# ECD-UY: Data resample and refilling example

This notebook shows how to fix irregular periods of records and fill gaps of data. For this example it is used the electric water heater subset of ECD-UY, specifically the data consumption of the appliance.

## Load the needed libraries

In [1]:
# python
import datetime
from typing import List, Tuple, Union
import glob
from os.path import join, exists
from os import makedirs
import yaml
import warnings
from datetime import datetime, timedelta

# data-science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ciso8601

%pylab inline
%matplotlib inline
plt.rcParams['figure.figsize'] = [18, 5]
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

print("Pandas version {}".format(pd.__version__))

Populating the interactive namespace from numpy and matplotlib
Pandas version 1.1.0


/Users/jp/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## Download the subset

In [2]:
!curl -O https://url-to-repo/ecd-uy/electric-water-heater-subset.tar.gz
!tar xzf electric-water-heater.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: url-to-repo
tar: Error opening archive: Failed to open 'electric-water-heater.tar.gz'


## Declare the path where the files are located

In [3]:
BASE_DATA_SOURCE = "../../DATOS-CARACTERIZACION-UTE-STANDARDIZED"
# BASE_DATA_SOURCE = "./electric-water-heater-subset"

## Load the appliance consumption data

In [4]:
filenames = glob.glob(BASE_DATA_SOURCE + '/appl_consumption_data_*.csv')

data_appl_consum = [pd.read_csv(
    fname,
) for fname in filenames]
data_appl_consum = pd.concat(data_appl_consum, ignore_index=True)

data_appl_consum.head()

,datetime,app_meter_id,value,voltage
0,2018-09-01 00:00:55-03:00,866131033542461,0.0,28
1,2018-09-01 00:00:57-03:00,866131033547650,0.0,223
2,2018-09-01 00:00:49-03:00,866131033550118,0.0,234
3,2018-09-01 00:00:55-03:00,866873020621150,4.2,0
4,2018-09-01 00:01:55-03:00,866131033542461,0.0,28


In [5]:
# Limit the records to year 2017 and parse the datetimes column
data_appl_consum_2017 = data_appl_consum[data_appl_consum.datetime.str.startswith("2017")]
data_appl_consum_2017['datetime'] = data_appl_consum_2017.datetime.apply(lambda x: ciso8601.parse_datetime(x))
data_appl_consum_2017.head()

,datetime,app_meter_id,value,voltage
171367,2017-07-01 00:00:55-03:00,10429800,0.0,227
171368,2017-07-01 00:00:16-03:00,10432617,0.0,248
171369,2017-07-01 00:00:06-03:00,10435950,0.0,257
171370,2017-07-01 00:00:22-03:00,10435972,0.0,259
171371,2017-07-01 00:00:56-03:00,10452106,0.0,178


## Resample of the data

Resample the data to periods of strictrly one minute, taking the maximum values in case of several values coincide in the same minute bin. If no values exist for the minute bin, it is left as NaN.

In [6]:
data_resampled = data_appl_consum_2017.groupby('app_meter_id').resample(
    '1min', on='datetime', origin='start_day'
).max()
data_resampled = data_resampled.drop(columns=['datetime', 'app_meter_id']).reset_index()
data_resampled.head()

,app_meter_id,datetime,value,voltage
0,466147,2017-09-13 18:50:00-03:00,0.0,0.0
1,466147,2017-09-13 18:51:00-03:00,0.0,266.0
2,466147,2017-09-13 18:52:00-03:00,1800.0,276.0
3,466147,2017-09-13 18:53:00-03:00,0.0,372.0
4,466147,2017-09-13 18:54:00-03:00,0.0,178.0


## Detecting gaps

In [29]:
gaps = data_resampled[~data_resampled.value.isna()].groupby('app_meter_id').datetime.diff()[
    lambda x: x > timedelta(minutes=1, seconds=0)
]
print(gaps.head(10))
print("Number of gaps detected: {}".format(gaps.shape[0]))

541    0 days 00:12:00
1096   0 days 00:02:00
1102   0 days 00:02:00
1266   0 days 00:15:00
1985   0 days 00:12:00
2713   0 days 00:18:00
3432   0 days 00:15:00
3999   0 days 00:02:00
4158   0 days 00:18:00
4880   0 days 00:18:00
Name: datetime, dtype: timedelta64[ns]
Number of gaps detected: 85893


For the records of the year 2017, 85.893 gaps were detected. Gaps are intervals greater than the period of one minutes. Next, many stats are calculated to have a better description of the characteristics of the gaps and determine when a gap is short or when it is long.

In [33]:
print(gaps.describe(percentiles=[.25, .5, .75, .9, .95]))
print("\nmedian\t{}".format(gaps.median()))

count                        85893
mean     0 days 00:57:28.361566134
std      0 days 18:53:16.580367471
min                0 days 00:02:00
25%                0 days 00:04:00
50%                0 days 00:12:00
75%                0 days 00:15:00
90%                0 days 00:19:00
95%                0 days 00:27:00
max              103 days 01:56:00
Name: datetime, dtype: object

median	0 days 00:12:00


Taking into account the calculated stats, the gap lenght of 12 minutes is taken as the threshold to classify between short and long gaps, i.e., if a gap length is lower than 12 minutes the gap is considered short, elsewhere, it is considered long.

### Detect long gaps

In [22]:
mask = data_resampled.copy()
def check_null_val(g):
    g['ones'] = 1  # column const to 1
    g['cumsum'] = g.value.notnull().cumsum()  # Increment only in presence of True, thus, increment only when value is not null
    g['long_gap'] = (g.groupby('cumsum').transform('count')['ones'] > 12) & g['value'].isnull()
    return g
mask = mask.groupby('app_meter_id').apply(check_null_val)

In [28]:
# Example of long gap
mask.iloc[1250:1270]

,app_meter_id,datetime,value,voltage,ones,cumsum,long_gap
1250,466147,2017-09-14 15:40:00-03:00,1800.0,236.0,1,1238,False
1251,466147,2017-09-14 15:41:00-03:00,1728.0,231.0,1,1239,False
1252,466147,2017-09-14 15:42:00-03:00,NaN,NaN,1,1239,True
1253,466147,2017-09-14 15:43:00-03:00,NaN,NaN,1,1239,True
1254,466147,2017-09-14 15:44:00-03:00,NaN,NaN,1,1239,True
1255,466147,2017-09-14 15:45:00-03:00,NaN,NaN,1,1239,True
1256,466147,2017-09-14 15:46:00-03:00,NaN,NaN,1,1239,True
1257,466147,2017-09-14 15:47:00-03:00,NaN,NaN,1,1239,True
1258,466147,2017-09-14 15:48:00-03:00,NaN,NaN,1,1239,True
1259,466147,2017-09-14 15:49:00-03:00,NaN,NaN,1,1239,True


The mask will be used next to interpolate ONLY the short gaps and set to zeto the long gaps.

## Refilling gaps

Short gaps will be refilled with an interpolation technique while the long gaps, interpreted as the appliance was shut down, will be refilled with zero value.

In [36]:
total_missed_values = mask[mask.value.isnull()].shape[0]
short_missed_values = mask[mask.value.isnull() & ~mask.long_gap].shape[0]
long_missed_values = mask[mask.value.isnull() & mask.long_gap].shape[0]
print(
    "A total of {} values are missed, "
    "where {} corresponds to short gaps and {} corresponds to long ones.".format(
        total_missed_values, short_missed_values, long_missed_values
    )
)

A total of 4850609 values are missed, where 282933 corresponds to short gaps and 4567676 corresponds to long ones.


In [43]:
data_resampled_refilled = data_resampled.copy()
data_resampled_refilled.value = data_resampled.value.interpolate(method='linear')
data_resampled_refilled.voltage = data_resampled.value.interpolate(method='linear')
data_resampled_refilled.value[mask.long_gap] = 0
data_resampled_refilled.voltage[mask.long_gap] = 0

In [54]:
# Validate short gap
print(mask[['app_meter_id', 'datetime', 'value', 'voltage']].iloc[525:535])
print(data_resampled_refilled.iloc[525:535])

     app_meter_id                  datetime   value  voltage
525        466147 2017-09-14 03:35:00-03:00  1776.0    567.0
526        466147 2017-09-14 03:36:00-03:00  1776.0    154.0
527        466147 2017-09-14 03:37:00-03:00  1800.0    191.0
528        466147 2017-09-14 03:38:00-03:00  1776.0   1457.0
529        466147 2017-09-14 03:39:00-03:00  1776.0    125.0
530        466147 2017-09-14 03:40:00-03:00     NaN      NaN
531        466147 2017-09-14 03:41:00-03:00     NaN      NaN
532        466147 2017-09-14 03:42:00-03:00     NaN      NaN
533        466147 2017-09-14 03:43:00-03:00     NaN      NaN
534        466147 2017-09-14 03:44:00-03:00     NaN      NaN
     app_meter_id                  datetime   value  voltage
525        466147 2017-09-14 03:35:00-03:00  1776.0   1776.0
526        466147 2017-09-14 03:36:00-03:00  1776.0   1776.0
527        466147 2017-09-14 03:37:00-03:00  1800.0   1800.0
528        466147 2017-09-14 03:38:00-03:00  1776.0   1776.0
529        466147 2017-0

In [44]:
# Validate the previous long gap
data_resampled_refilled.iloc[1250:1270]

,app_meter_id,datetime,value,voltage
1250,466147,2017-09-14 15:40:00-03:00,1800.0,1800.0
1251,466147,2017-09-14 15:41:00-03:00,1728.0,1728.0
1252,466147,2017-09-14 15:42:00-03:00,0.0,0.0
1253,466147,2017-09-14 15:43:00-03:00,0.0,0.0
1254,466147,2017-09-14 15:44:00-03:00,0.0,0.0
1255,466147,2017-09-14 15:45:00-03:00,0.0,0.0
1256,466147,2017-09-14 15:46:00-03:00,0.0,0.0
1257,466147,2017-09-14 15:47:00-03:00,0.0,0.0
1258,466147,2017-09-14 15:48:00-03:00,0.0,0.0
1259,466147,2017-09-14 15:49:00-03:00,0.0,0.0


In [45]:
# validate number of NaNs
data_resampled_refilled[data_resampled_refilled.value.isnull()].shape[0]

0

In [ ]:
# END